In [1]:
from ortools.linear_solver import pywraplp
from ortools.linear_solver import pywraplp
import numpy as np
from itertools import combinations
import pandas as pd
import numpy as np

In [2]:
dose_matrix = pd.read_csv('DoseMatrix.csv')

In [3]:
dose_matrix =np.array(dose_matrix)

In [4]:
CTV_index=[167, 168, 169, 170, 171, 172, 173,
      186, 187, 188, 189, 190, 191, 192, 193, 194,
      206, 207, 208, 209, 210, 211, 212, 213, 214, 
           227, 228, 229, 230, 231, 232, 233]

bladder_index=[87, 88, 89, 90, 91, 92, 93, 
         106, 107, 108, 109, 110, 111, 112, 113, 114, 
         126, 127, 128, 129, 130, 131, 132, 133, 134, 
             147, 148, 149, 150, 151, 152, 153]

Rectal_index=[248, 249, 250, 251, 252, 
        267, 268, 269, 270, 271, 272, 273, 
        287, 288, 289, 290, 291, 292, 293, 
              308, 309, 310, 311, 312]

Un_index=[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 
             21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 
             41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 
             61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 
             81, 82, 83, 84, 85, 86, 94, 95, 96, 97, 98, 99, 100, 101, 104, 105, 115, 118, 119, 120,
             125, 139, 140, 145, 146, 154, 159, 160, 165, 166, 174, 179, 180, 185, 199, 200, 205, 219, 220, 
             225, 226, 234, 239, 240, 241, 244, 245, 246, 247, 253, 254, 255, 258, 259, 260, 261, 262, 263, 
             264, 265, 266, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 294, 295, 296, 
             297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 313, 314, 315, 316, 317, 318, 319, 320, 
             321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 
             340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 
             359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 
             378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 
             397, 398, 399]

Left_index=[116, 117, 
       135, 136, 137, 138, 
       155, 156, 157, 158, 
       175, 176, 177, 178,
       195, 196, 197, 198, 
       215, 216, 217, 218, 
       235, 236, 237, 238, 
            256, 257]

Right_index=[101, 102, 
        120, 121, 122, 123, 
        140, 141, 142, 143, 
        160, 161, 162, 163, 
        180, 181, 182, 183, 
        200, 201, 202, 203, 
        220, 221, 222, 223, 
             241, 242]

In [5]:
len(CTV_index)+len(bladder_index)+len(Rectal_index)+len(Un_index)+len(Left_index)+len(Right_index)

400

In [6]:
solver = pywraplp.Solver('Final Project',
                         pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
objective=solver.Objective() 
objective.SetMinimization()

# Decision Variables
beamlets = []
for i in range(60):
    beamlets.append(solver.NumVar(0,solver.infinity(), 'beamlets[%d]' % i))
voxels = []    
for i in range(400):
    voxels.append(solver.NumVar(0,solver.infinity(),'voxels[%d]' % i))

    
#Constraints
voxel_constraints = [0 for _ in range(400)]
for i in range(400):
    voxel_constraints[i] = solver.Constraint(0,0)
    for j in range(60):
        voxel_constraints[i].SetCoefficient(beamlets[j],dose_matrix[i,j+1])
    voxel_constraints[i].SetCoefficient(voxels[i], -1)
        
######################################################## CTV#################################################

# every voxel receives a uniform dose of 82.8       ( = 82.8)
z_p = [0 for _ in range(len(CTV_index))]
z_n = [0 for _ in range(len(CTV_index))]

for i in range(len(CTV_index)):
    z_p[i] = solver.NumVar(0, solver.infinity(), "Z plus CTV %d" %i)
    z_n[i] = solver.NumVar(0, solver.infinity(), "Z minus CTV %d" %i)
     
CTV_constraint = [0 for _ in range(len(CTV_index))]
for i in range(len(CTV_index)):
    CTV_constraint[i] = solver.Constraint(82.8,82.8)
    
    for j in range(60):
        CTV_constraint[i].SetCoefficient(beamlets[j],dose_matrix[CTV_index[i],j+1])
    CTV_constraint[i].SetCoefficient(z_p[i],1)
    CTV_constraint[i].SetCoefficient(z_n[i],-1)

CTV_constraint_2 = [[0 for _ in range(len(CTV_index))]for _ in range(len(CTV_index))]
for i in range(len(CTV_index)):
    for j in range(len(CTV_index)):
        CTV_constraint_2[i][j] = solver.Constraint(0, 0.05*82.8)
        CTV_constraint_2[i][j].SetCoefficient(z_p[i],1)
        CTV_constraint_2[i][j].SetCoefficient(z_n[j],1.05)
        
        
    
############################################ bladder ####################################################

# # max dose to a voxel 81 ( < = 81)
z_b_1 = [0 for _ in range(len(bladder_index))]

for i in range(len(bladder_index)):
    z_b_1[i] = solver.NumVar(0, solver.infinity(), "z bladder #1 %d" %i)
    

bladder_constraint_1 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_1[i] = solver.Constraint(-solver.infinity(),81)
    for j in range(60):
        bladder_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_1[i].SetCoefficient(z_b_1[i],-1)


# Average dose should be <= 50     ( <= 50)

z_b_2 = solver.NumVar(0, solver.infinity(), "z bladder #2 %d" %i)
bladder_constraint_2 = solver.Constraint(-solver.infinity(), 50)
bladder_constraint_2.SetCoefficient(z_b_2, -1)

for i in bladder_index:
    bladder_constraint_2.SetCoefficient(voxels[i],1/len(bladder_index))

        
    
# at most 10% of the bladder should receive a dose >65          ( >65)
y = [0 for _ in range(len(bladder_index))]
z_y = solver.IntVar(0,4, "z y constraint bladder")

for i in range(len(bladder_index)):
     y[i] = solver.IntVar(0,1,"y bladder %d" %i)

bladder_constraint_3 = [0 for _ in range(len(bladder_index))]
for i in range(len(bladder_index)):
    bladder_constraint_3[i] = solver.Constraint(-solver.infinity(),65)
    for j in range(60):
        bladder_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[bladder_index[i],j+1])
    bladder_constraint_3[i].SetCoefficient(y[i],-100)

bladder_constraint_3b = solver.Constraint(-solver.infinity(),0.1*len(bladder_index))
for j in range(len(bladder_index)):
     bladder_constraint_3b.SetCoefficient(y[j],1)
bladder_constraint_3b.SetCoefficient(z_y, -1)

        
        

        
# ####################### Rectal########################################

#max dose to a voxel: 79.2              ( < = 79.2)

z_r_1 = [0 for _ in range(len(Rectal_index))]

for i in range(len(Rectal_index)):
    z_r_1[i] = solver.NumVar(0, solver.infinity(), "z rectal #1 %d" %i)
    

Rectal_constraint_1 = [0 for _ in range(len(Rectal_index))]
for i in range(len(Rectal_index)):
    Rectal_constraint_1[i] = solver.Constraint(-solver.infinity(),79.2)
    for j in range(60):
        Rectal_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Rectal_index[i],j+1])
    Rectal_constraint_1[i].SetCoefficient(z_r_1[i],-1)

    
    
# #average dose should be <= 40

z_r_2 = solver.NumVar(0, solver.infinity(), "z rectal #2 %d" %i)
Rectal_constraint_2 = solver.Constraint(-solver.infinity(), 40)
Rectal_constraint_2.SetCoefficient(z_r_2, -1.0)

for i in Rectal_index:
    Rectal_constraint_2.SetCoefficient(voxels[i],1/len(Rectal_index))



# ############################## Unspecified ############################

# max dose to a voxel: 72         ( < =72)

z_u_1 = [0 for _ in range(len(Un_index))]

for i in range(len(Un_index)):
    z_u_1[i] = solver.NumVar(0, solver.infinity(), "z Unspecified #1 %d" %i)
    

Un_constraint_1 = [0 for _ in range(len(Un_index))]
for i in range(len(Un_index)):
    Un_constraint_1[i] = solver.Constraint(-solver.infinity(),72)
    for j in range(60):
        Un_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Un_index[i],j+1])
    Un_constraint_1[i].SetCoefficient(z_u_1[i],-1)


#############################Left femur head ##########################

#max dose to a voxel: 50            ( <= 50)
  
z_l_1 = [0 for _ in range(len(Left_index))]

for i in range(len(Left_index)):
    z_l_1[i] = solver.NumVar(0, solver.infinity(), "z left femur #1 %d" %i)
    

left_constraint_1 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    left_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        left_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    left_constraint_1[i].SetCoefficient(z_l_1[i],-1)    
        

        
# at most 10% of the left femur should receive a dose > 40 
y_left = [0 for _ in range(len(Left_index))]
z_y_left = solver.IntVar(0,4, "z y constraint left femur")

for i in range(len(Left_index)):
     y_left[i] = solver.IntVar(0,1,"y left femur %d" %i)

Left_constraint_3 = [0 for _ in range(len(Left_index))]
for i in range(len(Left_index)):
    Left_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Left_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Left_index[i],j+1])
    Left_constraint_3[i].SetCoefficient(y_left[i],-100)

Left_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Left_index))
for j in range(len(Left_index)):
     Left_constraint_3b.SetCoefficient(y_left[j],1)
Left_constraint_3b.SetCoefficient(z_y_left,-1)


############################# right femur head ##########################


#max dose to a voxel: 50               ( <= 50 )
z_rl_1 = [0 for _ in range(len(Right_index))]

for i in range(len(Right_index)):
    z_rl_1[i] = solver.NumVar(0, solver.infinity(), "z right femur #1 %d" %i)
    

Right_constraint_1 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    Right_constraint_1[i] = solver.Constraint(-solver.infinity(),50)
    for j in range(60):
        Right_constraint_1[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    Right_constraint_1[i].SetCoefficient(z_rl_1[i],-1)    
        

        
# at most 10% of the right femur should receive a dose > 40 
y_right = [0 for _ in range(len(Right_index))]
z_y_right = solver.IntVar(0,4, "z y constraint right femur")

for i in range(len(Right_index)):
     y_right[i] = solver.IntVar(0,1,"y right femur %d" %i)

Right_constraint_3 = [0 for _ in range(len(Right_index))]
for i in range(len(Right_index)):
    
    # <= 40
    Right_constraint_3[i] = solver.Constraint(-solver.infinity(),40)
    for j in range(60):
        Right_constraint_3[i].SetCoefficient(beamlets[j],dose_matrix[Right_index[i],j+1])
    # Big M constraint
    Right_constraint_3[i].SetCoefficient(y_right[i],-100)

Right_constraint_3b = solver.Constraint(-solver.infinity(),0.15*len(Right_index))
for j in range(len(Right_index)):
     Right_constraint_3b.SetCoefficient(y_right[j],1)
Right_constraint_3b.SetCoefficient(z_y_right, -1)
        
        
########################## Objective Function ###########################

# CTV
for i in range(len(z_p)):
    objective.SetCoefficient(z_p[i],2000)
    objective.SetCoefficient(z_n[i],2000)

# Bladders
for i in range(len(z_b_1)):
    objective.SetCoefficient(z_b_1[i],1)

objective.SetCoefficient(z_b_2,1)

# Rectal Solids
for i in range(len(z_r_1)):
    objective.SetCoefficient(z_r_1[i],1000)
    
objective.SetCoefficient(z_r_2,1)

#Unspecified
for i in range(len(z_u_1)):
    objective.SetCoefficient(z_u_1[i],1)

# left femur head
for i in range(len(z_l_1)):
    objective.SetCoefficient(z_l_1[i],800)

#right femur head
for i in range(len(z_rl_1)):
    objective.SetCoefficient(z_rl_1[i],800)

# bladder, left femur, right femur's at most constraint
objective.SetCoefficient(z_y,1)
objective.SetCoefficient(z_y_left,1)
objective.SetCoefficient(z_y_right,1)

status = solver.Solve()

if status == solver.OPTIMAL:
    print('Problem solved in %f milliseconds' %solver.wall_time())
elif status == solver.FEASIBLE:
    print('Solver claims feasibility but not optimality')
else:
    print('Solver ran to completion but did not find an optimal solution')

print('weighted slack total',objective.Value()) 


#print(bladder_constraint_2.dual_value())

#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

Problem solved in 20786.000000 milliseconds
weighted slack total 116047.48093794781


In [7]:
print(bladder_constraint_2.dual_value())

0.0


In [8]:
#print(solver.ExportModelAsLpFormat(False).replace('\\', '').replace(',_', ','), sep='\n')

## All Voxels

In [9]:
All_voxels = np.zeros((400)).astype(np.float)
for i in range(400):
    All_voxels[i] = voxels[i].solution_value()
#print(All_voxels.reshape(20,20))
All_voxels = pd.DataFrame(All_voxels.reshape((20,20)), index=[i for i in range(1,21)],
                          columns = [i for i in range(1,21)]).round(2)
All_voxels

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
1,88.38,80.82,66.92,50.62,36.43,25.57,16.92,10.45,6.79,6.17,8.44,12.20,17.21,24.50,29.98,28.35,23.01,20.48,22.12,25.69
2,81.08,80.82,74.34,62.88,49.44,36.48,25.15,16.54,11.14,9.23,11.16,15.74,22.20,30.61,35.86,32.93,26.69,24.27,25.81,28.07
3,69.49,74.37,74.38,69.54,60.41,48.10,35.02,24.43,17.71,15.03,16.88,22.03,28.96,36.95,41.03,37.08,30.53,27.67,27.83,27.99
4,56.03,64.21,68.74,69.69,66.44,57.86,45.36,33.75,26.51,24.30,26.71,31.42,36.65,42.70,45.68,41.29,33.94,29.45,27.53,25.51
5,42.40,52.54,59.97,64.90,66.81,63.42,54.34,44.06,37.65,36.81,39.89,42.90,44.86,48.53,50.60,45.18,35.82,29.06,25.05,20.94
6,29.74,40.68,50.00,57.54,63.07,64.49,60.68,54.75,50.97,51.08,53.57,54.57,54.11,55.89,56.08,47.98,35.78,26.71,20.70,15.01
7,18.90,29.40,39.88,49.45,57.72,63.18,65.00,65.00,64.69,64.52,65.00,65.00,64.41,65.00,62.07,50.00,34.41,22.81,15.11,9.12
8,10.55,19.33,30.10,41.72,53.16,62.71,69.39,73.82,75.54,74.57,73.63,73.63,73.85,74.11,68.70,52.58,32.80,18.27,9.69,4.66
9,5.05,11.36,21.45,35.22,51.09,65.36,74.99,80.01,81.26,80.38,80.01,80.01,80.06,80.87,75.38,56.53,32.49,15.05,6.10,2.20
10,2.26,6.57,15.79,31.60,52.10,70.38,80.39,83.06,82.80,82.80,83.75,83.49,82.66,84.09,80.01,61.00,34.98,15.57,5.72,1.77


In [10]:
OUTPUT  = 'output/cross_section_p4.csv'
cross_section = All_voxels.to_csv(OUTPUT, index=[i for i in range(1,21)], columns = [i for i in range(1,21)])

## Check Constraint

**Beamlets Check**

In [11]:
# check beamlets value
beamlets_value = []
for i in range(60):
    beamlets_value.append(beamlets[i].solution_value())
print('Beamlets values are ')
print(np.array(beamlets_value))   

Beamlets values are 
[16.96397438  0.          0.          6.89328361  0.          0.
 27.96771716  2.75106494  0.         64.61330867  0.         49.53923886
  6.87758631  0.          4.34038406  0.          0.          0.
 41.39469493  0.          0.          0.         14.76014386  0.
 12.16804208  0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.          0.          0.          0.          0.          0.
  0.         32.46796111 29.9978877  24.33048916 24.8790603  14.45125104
  0.          0.          0.          4.81328824 54.60515914  0.
 26.55060377 28.39401859  0.         14.4476048   9.72414851  0.        ]


**CTV check**

In [12]:
#Every voxel receives a uniform dose of 82.8 
CTV_voxel = []
every = 0
for i in CTV_index:
    every = voxels[i].solution_value() 
    CTV_voxel.append(every)
print('CTV voxel dose(Gy)')
print(np.array(CTV_voxel))

CTV voxel dose(Gy)
[80.01415603 81.2628027  80.38481993 80.01415603 80.01415603 80.06327075
 80.87328887 80.38916355 83.0587902  82.8        82.8        83.75458152
 83.48814641 82.66243596 84.08967234 80.01415494 82.8        83.78985768
 82.83323864 82.90464546 84.08967137 83.90272397 82.8        84.08967241
 81.00035059 82.8        82.38849017 81.47603212 81.88646582 82.04569036
 81.22282117 82.13037703]


In [13]:
# the distance to 82.8
zp0 = np.array([[x.solution_value() for x in z_p]])
zn0 = np.array([[x.solution_value() for x in z_n]])
print('Positive distance to 82.8 \n',zp0)
print('\n')
print('Negative distance to 82.8 \n',zn0)

Positive distance to 82.8 
 [[2.78584397e+00 1.53719730e+00 2.41518007e+00 2.78584397e+00
  2.78584397e+00 2.73672925e+00 1.92671113e+00 2.41083645e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 1.37564038e-01 0.00000000e+00 2.78584506e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00
  1.79964941e+00 3.65418564e-14 4.11509829e-01 1.32396788e+00
  9.13534176e-01 7.54309639e-01 1.57717883e+00 6.69622971e-01]]


Negative distance to 82.8 
 [[0.         0.         0.         0.         0.         0.
  0.         0.         0.2587902  0.         0.         0.95458152
  0.68814641 0.         1.28967234 0.         0.         0.98985768
  0.03323864 0.10464546 1.28967137 1.10272397 0.         1.28967241
  0.         0.         0.         0.         0.         0.
  0.         0.        ]]


In [14]:
print('maximum difference of CTV voxels values to 82.8 are ', (max(CTV_voxel)-82.8)/82.8, (82.8- min(CTV_voxel))/82.8)

maximum difference of CTV voxels values to 82.8 are  0.015575753707711934 0.03364547173267548


In [15]:
print('maximum CTV voxels value is ', max(CTV_voxel))

maximum CTV voxels value is  84.08967240699855


In [16]:
print('minimum CTV voxels value is ', min(CTV_voxel))

minimum CTV voxels value is  80.01415494053447


In [17]:
print('percentage of voxels below 79:', sum((np.array(CTV_voxel) < 79)*1) / len(CTV_voxel))

percentage of voxels below 79: 0.0


In [18]:
print('percentage of voxels Above 86.94:', sum((np.array(CTV_voxel) > 86.94)*1) / len(CTV_voxel))

percentage of voxels Above 86.94: 0.0


**Bladder Check**

In [19]:
# Max dose to a voxel: 81
bladder_voxel = []
every = 0
for i in bladder_index:
    every = round(voxels[i].solution_value(),4)
    bladder_voxel.append(every)
print('Bladder Voxel Doses\n',np.array(bladder_voxel))

Bladder Voxel Doses
 [44.0602 37.6515 36.8131 39.8919 42.8998 44.8601 48.5276 60.6756 54.7458
 50.9677 51.0834 53.5656 54.5737 54.1064 55.8893 56.082  65.     65.
 64.6917 64.5155 65.     65.     64.4126 65.     62.0651 73.8204 75.5391
 74.571  73.6314 73.6274 73.853  74.1057]


In [20]:
# if any voxel has more than 81
zb1 = np.array([[x.solution_value() for x in z_b_1]])
print('distances to 81 if voxel value is above 81\n',zb1)

distances to 81 if voxel value is above 81
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0.]]


In [21]:
# check if there are voxel with value more than 81
print('are there values more than 81?')
print(max(bladder_voxel) > 81)

print('how many percentage of voxel values are more than 81?')

print(round(sum((np.array(bladder_voxel) >81)*1)/len(bladder_voxel),2)*100,'%')

are there values more than 81?
False
how many percentage of voxel values are more than 81?
0.0 %


In [22]:
# average dose shouldbe <= 50
print('average dose received by bladder voxels are ',sum(bladder_voxel)/len(bladder_voxel))
print('slack variable to bring down the value to below 50: ', z_b_2.solution_value())

average dose received by bladder voxels are  58.944581250000006
slack variable to bring down the value to below 50:  8.944580013488945


In [23]:
# at most 10% of the bladder should receive a dose > 65
y_value = np.array([t.solution_value() for t in y])
print('voxel that are larger that 65 are indicated by 1 \n',y_value)

voxel that are larger that 65 are indicated by 1 
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1.]


In [24]:
test = np.array(list(map(lambda x: 1 if x > 65.00 else 0,bladder_voxel)))
print(round(sum(test)/len(test),4), ' of the voxels are above 65')

0.2188  of the voxels are above 65


In [25]:
print('voxel that are larger that 65 are indicated by 1 \n',test)

voxel that are larger that 65 are indicated by 1 
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1]


In [26]:
y_value == test

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True])

In [27]:
print('the number of constraints we have to set loose to be above 65',z_y.solution_value())

the number of constraints we have to set loose to be above 65 4.0


**Rectal Solid Check**

In [28]:
# max dose to a voxel: 79.2
Rectal_voxel = []
every = 0
for i in Rectal_index:
    every = round(voxels[i].solution_value(),8)
    Rectal_voxel.append(every)
print('rectal voxel dose values')
print(np.array(Rectal_voxel))

rectal voxel dose values
[79.83368294 79.2        79.32232025 79.2        78.33539154 69.97952676
 73.11778124 75.66945274 77.37633246 76.8071187  75.61774726 78.21992967
 58.1731547  62.14402621 68.77651923 74.45870925 75.83660185 75.43704764
 79.2        48.9404677  57.15521186 68.00496889 75.39541422 78.79298424]


In [29]:
zr1 = np.array([[x.solution_value() for x in z_r_1]])
print('distances to 79.2 if voxel value is above 79.2\n',zr1)

distances to 79.2 if voxel value is above 79.2
 [[0.63368294 0.         0.12232025 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.        ]]


In [30]:
# check if there's value larger than 79.2
print('are there values more than 79.2?')
print(max(Rectal_voxel) > 79.2)

print('how many percentage of voxel values are more than 79.2?')

print(round(sum((np.array(Rectal_voxel) >79.2)*1)/len(Rectal_voxel),2)*100,'%')

are there values more than 79.2?
True
how many percentage of voxel values are more than 79.2?
8.0 %


In [31]:
# average dose should be <= 40
print('average dose received by bladder voxels are ',sum(Rectal_voxel)/len(Rectal_voxel))
print('slack variable to bring down the value to below 40: ',z_r_2.solution_value())

average dose received by bladder voxels are  72.70809955625
slack variable to bring down the value to below 40:  32.70809955660276


**Unspecified**

In [32]:
# Max dose to a voxel: 72
Un_voxel = []
every = 0
for i in Un_index:
    every = round(voxels[i].solution_value(),3)
    Un_voxel.append(every)
np.array(Un_voxel)

array([ 88.38 ,  80.819,  66.924,  50.621,  36.426,  25.571,  16.92 ,
        10.454,   6.791,   6.166,   8.437,  12.203,  17.208,  24.504,
        29.98 ,  28.353,  23.013,  20.482,  22.118,  25.693,  81.077,
        80.821,  74.337,  62.88 ,  49.444,  36.475,  25.15 ,  16.536,
        11.137,   9.227,  11.162,  15.736,  22.201,  30.611,  35.864,
        32.925,  26.694,  24.267,  25.808,  28.071,  69.488,  74.365,
        74.378,  69.536,  60.41 ,  48.101,  35.021,  24.43 ,  17.71 ,
        15.032,  16.883,  22.028,  28.957,  36.952,  41.031,  37.085,
        30.525,  27.667,  27.827,  27.987,  56.035,  64.214,  68.741,
        69.687,  66.443,  57.86 ,  45.36 ,  33.747,  26.508,  24.301,
        26.714,  31.422,  36.655,  42.702,  45.68 ,  41.293,  33.941,
        29.45 ,  27.532,  25.511,  42.401,  52.538,  59.965,  64.898,
        66.813,  63.422,  54.341,  50.604,  45.176,  35.819,  29.057,
        25.054,  20.941,  29.745,  40.678,  63.068,  64.495,  47.979,
        20.696,  15.

In [33]:
print('Are there voxel values more than 72?')
max(Un_voxel) > 72

Are there voxel values more than 72?


True

In [34]:
print('how many percentage of voxel values are more than 72?')

print(round(sum((np.array(Un_voxel) >72)*1)/len(Un_voxel),2)*100,'%')

how many percentage of voxel values are more than 72?
14.000000000000002 %


**Left Femur Head**

In [35]:
# max dose to be voxel 50
Left_voxel = []
every = 0
for i in Left_index:
    every = round(voxels[i].solution_value(),2)
    Left_voxel.append(every)
print('left femur head voxel values')
np.array(Left_voxel)

left femur head voxel values


array([35.78, 26.71, 50.  , 34.41, 22.81, 15.11, 52.58, 32.8 , 18.27,
        9.69, 56.53, 32.49, 15.05,  6.1 , 61.  , 34.98, 15.57,  5.72,
       64.11, 40.  , 20.8 ,  9.51, 64.85, 44.9 , 28.79, 17.38, 47.06,
       35.41])

In [36]:
zl1 = np.array([[x.solution_value() for x in z_l_1]])
print('distances to 79.2 if voxel value is above 50\n',zl1)

distances to 79.2 if voxel value is above 50
 [[ 0.          0.          0.          0.          0.          0.
   2.58446976  0.          0.          0.          6.53417715  0.
   0.          0.         10.99682563  0.          0.          0.
  14.11067142  0.          0.          0.         14.84830999  0.
   0.          0.          0.          0.        ]]


In [37]:
print('are there values more than 50?')
print(max(Left_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Left_voxel) >50)*1)/len(Left_voxel),2)*100,'%')

are there values more than 50?
True
how many percentage of voxel values are more than 50?
18.0 %


In [38]:
# at most 15% of the left femur head should receive > 40
y_left_value = np.array([t.solution_value() for t in y_left])
print('voxel that are larger that 40 are indicated by 1 \n',y_left_value)

voxel that are larger that 40 are indicated by 1 
 [0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 1. 1.
 0. 0. 1. 0.]


In [39]:
test_left_femur = np.array(list(map(lambda x: 1 if x >40.00 else 0,Left_voxel)))
print(round(sum(test_left_femur)/len(test_left_femur),4), ' of the left femur voxels are above 40')

0.2857  of the left femur voxels are above 40


In [40]:
print('voxel that are larger that 40 are indicated by 1 \n',test_left_femur)

voxel that are larger that 40 are indicated by 1 
 [0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 1 1 0 0 1 0]


In [41]:
test_left_femur == y_left_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [42]:
print('the number of constraints we have to set loose to be above 40',z_y_left.solution_value())

the number of constraints we have to set loose to be above 40 4.0


**Right Femur Head**

In [43]:
# max dose to a voxel: 50 
Right_voxel = []
every = 0
for i in Right_index:
    every = round(voxels[i].solution_value(),2)
    Right_voxel.append(every)
print("right femur head voxel values")
np.array(Right_voxel)

right femur head voxel values


array([40.68, 50.  , 18.9 , 29.4 , 39.88, 49.45, 10.55, 19.33, 30.1 ,
       41.72,  5.05, 11.36, 21.45, 35.22,  2.26,  6.57, 15.79, 31.6 ,
        1.89,  6.03, 15.65, 32.89,  4.33, 10.81, 22.43, 39.31, 21.4 ,
       34.42])

In [44]:
# slack variable needed to fullfill the constraints
zrl1 = np.array([[x.solution_value() for x in z_rl_1]])
print('distances to 79.2 if voxel value is above 50\n',zrl1)

distances to 79.2 if voxel value is above 50
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0.]]


In [45]:
# check if there's any right voxel value that is bigger than 50
print('are there values more than 50?')
print(max(Right_voxel) > 50)

print('how many percentage of voxel values are more than 50?')

print(round(sum((np.array(Right_voxel) >50)*1)/len(Right_voxel),2)*100,'%')

are there values more than 50?
False
how many percentage of voxel values are more than 50?
0.0 %


In [46]:
# at most 15% of the right  femur head should receive > 40
y_right_value = np.array([t.solution_value() for t in y_right])
print('voxel that are larger that 40 are indicated by 1 \n',y_right_value)

voxel that are larger that 40 are indicated by 1 
 [1. 1. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


In [47]:
test_right = np.array(list(map(lambda x: 1 if x > 40.00 else 0,Right_voxel)))
print(round(sum(test_right)/len(test_right),4), ' of the right femur voxels are above 40')

0.1429  of the right femur voxels are above 40


In [48]:
print('voxel that are larger that 40 are indicated by 1 \n',test_right)

voxel that are larger that 40 are indicated by 1 
 [1 1 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]


In [49]:
test_right == y_right_value

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True])

In [50]:
print('the number of constraints we have to set loose to be above 40',z_y_right.solution_value())

the number of constraints we have to set loose to be above 40 0.0
